# Movie Classification

In [ ]:
!conda install -c conda-forge keras --yes
!conda install -c anaconda tensorflow-gpu --yes
!conda install -c anaconda gensim --yes
!conda install -c spacy spacy --yes
!pip install kapre

Solving environment: - 

In [ ]:
%matplotlib inline
import cv2
import sys
import skvideo.io
from skvideo.measure import scenedet
import json
from scipy.signal import argrelextrema
import urllib
from urlparse import urlparse

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import time as t
import math
import seaborn as sns

In [ ]:
import sys
sys.path.append('lmtd')
from lmtd9 import LMTD
from lmtd9 import database as db
from lmtd9 import evaluation

In [ ]:
# Librosa for audio
import librosa
# And the display module for visualization
import librosa.display
import glob
import os

#import tensorflow as tf
from matplotlib.pyplot import specgram
from sklearn import preprocessing
#from keras.utils.np_utils import to_categorical
from tqdm import tqdm
from time import sleep

In [ ]:
from ffmpeg_load_audio import ffmpeg_load_audio
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

In [ ]:
import threading
import requests
import urlparse, os
import time

In [ ]:
import scipy

In [ ]:
import seaborn as sns
import pandas as pd
import operator
from collections import OrderedDict

In [ ]:
from multiprocessing import Pool
import multiprocessing
import time
from itertools import repeat

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from scipy import interp
from itertools import cycle

In [ ]:
import tensorflow as tf

In [ ]:
import os
os.environ['KERAS_BACKEND']='tensorflow'
from keras import utils
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras import applications
from keras.layers import GlobalAveragePooling2D, merge, Input
from keras.models import Model

In [ ]:
from keras import backend as K
from keras.engine.topology import Layer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import image
from keras.utils import to_categorical
from keras.layers import TimeDistributed, BatchNormalization
from keras.layers import Input, Convolution1D, GlobalMaxPooling1D, merge, Dense, Dropout
from keras.layers import Activation, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import Adam

In [ ]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import datetime

In [ ]:
from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D
from kapre.augmentation import AdditiveNoise

In [ ]:
nb_epoch = 50,
batch_size = 64
learning_rate = 0.001
gaussian_noise_lstm = 0.01
lstm_size = 512
gru_size = 512
recurrent_dropout = 0.2
dropout_cnn = 0.5
gaussian_noise_cnn = 0.1
model_output_path = "/home/paulo/mestrado/MovieGenreClassifier.model"
validation_split = 0.2
test_split = 0.2
regularizer_lambda = 0.01
GLOVE_DIR = "/home/paulo/mestrado/glove.6B/"
min_word_frequency_word2vec = 5,
embed_size_word2vec = 300
context_window_word2vec = 10
MAX_NB_WORDS = 50000
max_sentence_len = 300
maxlen = max_sentence_len
min_sentence_length = 15
sr = 44100 #sampling rate 44.1KHz
audio_channels = 2
width = 299 #for inceptionV3
height = 299 #for inceptionV3   
depth = 3 #rgb
 #stereo
audio_input_shape = (audio_channels, sr) #2-channels - 44.1KHz
time_steps = 240
nb_features = 2048
nb_classes = 9
conv_filters = 384
dropout = 0.5
max_epochs = 10

n_cpus = multiprocessing.cpu_count()    
print('n_cpus:',n_cpus)      
number_gpus = 1
number_cpus = n_cpus-2

In [ ]:
lmtd = LMTD()

In [ ]:
'''
def setup_word2vec(all_data):    
    wordvec_model = Word2Vec(all_data, min_count=min_word_frequency_word2vec, size=embed_size_word2vec, window=context_window_word2vec)
    return wordvec_model
'''

In [ ]:
def setup_tokenizer(all_data):
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(all_data)
    return tokenizer

In [ ]:
def setup_glove_embeddings():
    embeddings_index = {}
    glove_file = 'glove.6B.{}d.txt'.format(embed_size_word2vec) #need to make both to match (embed_size_word2vec and globe index)
    f = open(os.path.join(GLOVE_DIR, glove_file))
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index

In [ ]:
def setup_embeddings_matrices(embeddings_index, word_index):
    embedding_matrix = np.random.random((len(word_index) + 1, embed_size_word2vec))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [ ]:
def setup_embedding_layer(word_index, embedding_matrix):
    embedding_layer = Embedding(len(word_index) + 1,
                                embed_size_word2vec,
                                weights=[embedding_matrix],
                                input_length=max_sentence_len,
                                trainable=True)
    return embedding_layer

In [ ]:
def createHistograms(img):
    hsv_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    hue, sat, ilum = hsv_image[:,:,0], hsv_image[:,:,1], hsv_image[:,:,2]    
    hist_hue, bin_edges_hue = np.histogram(np.ndarray.flatten(hue), bins=8)
    hist_sat, bin_edges_sat = np.histogram(np.ndarray.flatten(sat), bins=4)
    hist_ilum, bin_edges_ilum = np.histogram(np.ndarray.flatten(ilum), bins=4)
    return np.hstack((hist_hue, hist_sat, hist_ilum))

In [ ]:
def histogram_intersection(previousHist, currentHist):
    s_i = 0
    #for each bin 
    #print(currentHist.shape[0])
    for j in range(currentHist.shape[0]):
        min_bin_s_i = min(currentHist[j],previousHist[j])
        #print('minimum from bin {}: {}'.format(j,min_bin_s_i))
        min_sum_prev = sum(previousHist)
        min_sum_cur = sum(currentHist)
        min_both = min(min_sum_prev, min_sum_cur) #normalize (maximum can be 1)
        s_i+= (min_bin_s_i/float(min_both)) 
    return s_i

In [ ]:
def gradE(sNext, sCurrent):
    return sNext-sCurrent

def gradW(sPrevious, sCurrent):
    return sPrevious-sCurrent

def gaussianKernel(x, k=0.1):
    return np.exp(-(x/k)**2)

def smoothed_s(s_i_previous, s_i, s_i_next, lamb=0.1, k=0.1):
    gEi = gradE(s_i_next, s_i)
    gWi = gradW(s_i_previous, s_i)
    CEt = gaussianKernel(np.absolute(gEi))
    CWt = gaussianKernel(np.absolute(gWi))
    stnext_i = s_i + lamb*(CEt*gEi + CWt * gWi)
    return stnext_i

In [ ]:
target_image = 'images/paulo_meire.png'
img = cv2.imread(target_image)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()

In [ ]:
target_image2 = 'images/paulo_meire2.png'
img2 = cv2.imread(target_image2)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
plt.imshow(img2)
plt.show()

In [ ]:
hist1 = createHistograms(img)
plt.hist(hist1)
print(hist1)
hist2 = createHistograms(img2)
plt.hist(hist2)
print(hist2)
print(histogram_intersection(hist1, hist2))

# Color variance

In [ ]:
def color_variance(videodata, smooth_scene_cuts):
    Ls = []
    Us = []
    Vs = []
    for frame in smooth_scene_cuts:
        luv_image = cv2.cvtColor(frame, cv2.COLOR_RGB2Luv)
        L, u, v = luv_image[:,:,0], luv_image[:,:,1], luv_image[:,:,2]
        Ls.append(L)
        Us.append(u)
        Vs.append(v)        
    Luv = np.stack((np.array(Ls).flatten(), np.array(Us).flatten(), np.array(Vs).flatten()), axis=0)
    pCov = np.cov(Luv)
    #print('covLuv:', pCov)
    #print('varL:',np.var(np.array(Ls)))
    #print('varU:',np.var(np.array(Us)))
    #print('varV:',np.var(np.array(Vs)))
    totalColorVariance = np.linalg.det(pCov)
    #print('totalColorVariance:', totalColorVariance)
    return totalColorVariance

## Get Samples

In [ ]:
trailer_ids = lmtd.train_ids
comedy_id = None
horror_id = None
action_id = None

for _id in trailer_ids:
    # Returns a dictionary in which keys are the queried trailer_ids
    movie_data = lmtd.get_data_by_trailer_ids(_id)
    #print(movie_data)
    for metadata in movie_data.keys():
        print('trailer id : ', metadata)
        print('title db : ', movie_data[metadata]['Title'])
        print('genres db : ', movie_data[metadata]['Genre'])
        print('labels       : ', lmtd.train_labels[int(metadata)])
        print('re-converted : ', lmtd.binary_label_to_genre(lmtd.train_labels[int(metadata)])[0])
        if comedy_id is not None and horror_id is not None and action_id is not None:
            break  
        elif ('Comedy' in movie_data[metadata]['Genre']):
            comedy_id = _id
        elif ('Horror' in movie_data[metadata]['Genre']):    
            horror_id = _id            
        elif ('Action' in movie_data[metadata]['Genre']):    
            action_id = _id 
print('Comedy:',comedy_id)
print('Horror:',horror_id)
print('Action:',action_id)

# Lighting Key

In [ ]:
def lighting_key(videodata, smooth_scene_cuts):    
    hsv_images = []
    for frame in smooth_scene_cuts:
        hsv_image = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
        hsv_images.append(hsv_image)        
    meanHsv = np.mean(hsv_images)
    stdHsv = np.std(hsv_images)
    lightingkey = meanHsv * stdHsv
    #print(lightingkey)
    return lightingkey

# Motion Content

In [ ]:
def motion_content(videodata, smooth_scene_cuts, w=3):    
    frameOld = smooth_scene_cuts[0]     
    count_active_pixels = 0
    count_all_pixels = 0
    for currentFrame in smooth_scene_cuts:
        Hx = cv2.Sobel(currentFrame,cv2.CV_64F,1,0,ksize=w)
        #print('Hx:',Hx)
        Ht = currentFrame - frameOld  #temporal derivative
        #print('Ht:',Ht)
        Hx2 = Hx * Hx
        #print('Hx2:',Hx2)
        Ht2 = Ht * Ht
        #print('Ht2:',Ht2)
        HxHt = Hx * Ht
        #print('HxHt:',HxHt)
        Jxx = sum(Hx2)
        #print('Jxx:',Jxx)
        Jxt = sum(HxHt)
        #print('Jxt:',Jxt)
        Jtt = sum(Ht2)
        #print('Jtt:',Jtt)
        #print(2*Jxt/(Jxx-Jxt))
        theta = (1/2.0)*np.degrees(np.arctan(2*Jxt/(Jxx-Jxt)))
        count_all_pixels+= theta.size  
        #print(count_all_pixels)
        aux = theta[~np.isnan(theta)]
        aux = aux.flatten()
        #print(aux)
        count_active_pixels += aux[(0.0 < aux) | (aux < 10.0)].size
        #print(count_active_pixels)
        frameOld = currentFrame        
    return count_active_pixels/float(count_all_pixels)

In [ ]:
comedy_filename = 'trailers/videos/'+comedy_id+'.mp4'
horror_filename = 'trailers/videos/'+horror_id+'.mp4'
action_filename = 'trailers/videos/'+action_id+'.mp4'

In [ ]:
comedy_videometadata = skvideo.io.ffprobe(comedy_filename)
print(comedy_videometadata.keys())
print(json.dumps(comedy_videometadata["video"], indent=4))
print(json.dumps(comedy_videometadata["audio"], indent=4))

In [ ]:
comedy_videodata = skvideo.io.vread(comedy_filename)
print(comedy_videodata.shape)
T, M, N, C = comedy_videodata.shape

print("Number of frames: %d" % (T,))
print("Number of rows: %d" % (M,))
print("Number of cols: %d" % (N,))
print("Number of channels: %d" % (C,))

frame_rate = comedy_videometadata['video']['@avg_frame_rate']
num_frames = np.int(comedy_videodata.shape[0])
width = np.int(comedy_videometadata['video']['@width'])
height = np.int(comedy_videometadata['video']['@height'])

In [ ]:
# using the "luminance" algorithm
comedy_scene_lum_idx = skvideo.measure.scenedet(comedy_videodata, method='histogram', parameter1=1.0)
comedy_scenecuts = comedy_videodata[comedy_scene_lum_idx]

In [ ]:
print(len(comedy_scenecuts))

In [ ]:
def shot_detection(videodata):
    #videodata = skvideo.io.vread(movie_filename)
    #T, M, N, C = videodata.shape
    #videometadata = skvideo.io.ffprobe(movie_filename)
    #frame_rate = videometadata['video']['@avg_frame_rate']
    #num_frames = np.int(videodata.shape[0])
    #width = np.int(videometadata['video']['@width'])
    #height = np.int(videometadata['video']['@height'])
    numFrames, height, width, channels = videodata.shape
    smooth_list = []
    for t in range(2, numFrames-1):
        beforePrevHist = createHistograms(videodata[t-2])
        prevHist = createHistograms(videodata[t-1])
        currHist = createHistograms(videodata[t])
        nextHist = createHistograms(videodata[t+1])
        s_i_prev = histogram_intersection(beforePrevHist,prevHist)
        s_i_cur = histogram_intersection(prevHist,currHist)
        s_i_next = histogram_intersection(currHist,nextHist)  
        s_smooth = smoothed_s(s_i_prev, s_i_cur, s_i_next)
        smooth_list.append(s_smooth)    
    #if first derivative very near 0 (critical point) and second derivative > 0 (local minima)    
    local_minima_index = argrelextrema(np.array(smooth_list), np.less)
    #print(local_minima_index)
    #intersects with normal histogram method ()
    threshold_index = [i for i,v in enumerate(smooth_list) if v < 0.7]
    cut_scene_index = sorted(list(set(local_minima_index[0]).intersection(threshold_index)))  
    #print('Shots detected:', len(cut_scene_index))
    #print('Index cut scenes:', cut_scene_index)
    #plt.ylabel('Shot Detection')
    #plt.xlabel('Frames')
    #plt.show()   
    smooth_scene_cuts = videodata[cut_scene_index]
    return videodata, smooth_scene_cuts

def average_shot_length(videodata, smooth_scene_cuts):
    num_frames = np.int(videodata.shape[0])
    return num_frames/float(len(smooth_scene_cuts))

## Traditional Histogram Cut

In [ ]:
for frame in comedy_scenecuts:
    imgplot = plt.imshow(frame)
    plt.show()

## Smoothed Histogram Cut

In [ ]:
comedy_videodata, comedy_smooth_scene_cuts = shot_detection(comedy_videodata)
for frame in comedy_smooth_scene_cuts:
    imgplot = plt.imshow(frame)
    plt.show()

In [ ]:
'''
horror_videodata, horror_smooth_scene_cuts = shot_detection(horror_filename)
for frame in horror_smooth_scene_cuts:
    imgplot = plt.imshow(frame)
    plt.show()
'''

In [ ]:
'''
action_videodata, action_smooth_scene_cuts = shot_detection(action_filename)
for frame in action_smooth_scene_cuts:
    imgplot = plt.imshow(frame)
    plt.show()
'''

## Average Shot Length 

In [ ]:
print('Comedy Average shot length:', average_shot_length(comedy_videodata, comedy_smooth_scene_cuts))
#print('Horror Average shot length:', average_shot_length(horror_videodata, horror_smooth_scene_cuts))

## Color Variance

In [ ]:
print('Comedy Color Variance:', color_variance(comedy_videodata, comedy_smooth_scene_cuts))
#print('Horror Color Variance:', color_variance(horror_videodata, horror_smooth_scene_cuts))

# Lighting Key

### High value - comedy
### Low value - horror

In [ ]:
print('Comedy Lighting Key:', lighting_key(comedy_videodata, comedy_smooth_scene_cuts))
#print('Horror Lighting Key:', lighting_key(horror_videodata, horror_smooth_scene_cuts))

In [ ]:
print('Comedy Motion Content:', motion_content(comedy_videodata, comedy_smooth_scene_cuts))

In [ ]:
#print('Horror Motion Content:', motion_content(horror_videodata, horror_smooth_scene_cuts))

In [ ]:
#print('Action Motion Content:', motion_content(action_videodata, action_smooth_scene_cuts))

# Audio

In [ ]:
def extract_audio_features(audio_path):    
    y, sample_rate = librosa.load(audio_path)
    S = librosa.feature.melspectrogram(y, sr=sample_rate, n_mels=128)
    log_S = librosa.amplitude_to_db(S, ref=np.max)
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    tempo_bpm, beats = librosa.beat.beat_track(y=y_percussive, sr=sample_rate)
    stft = np.abs(librosa.stft(y))
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sample_rate)[0]
    spec_bandwith = librosa.feature.spectral_bandwidth(y=y, sr=sample_rate)[0]
    contrast = np.mean(librosa.feature.spectral_contrast(S=S, sr=sample_rate))
    flatness = librosa.feature.spectral_flatness(y=y)[0]
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sample_rate)[0]
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y)[0]
    mfcc        = librosa.feature.mfcc(y=y, sr=sample_rate, n_mfcc=40)
    mfccs = np.mean(mfcc.T,axis=0)
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(y, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=y_harmonic,sr=sample_rate).T,axis=0)
    hop_length = 512
    oenv = librosa.onset.onset_strength(y=y, sr=sample_rate, hop_length=hop_length)
    tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr=sample_rate, hop_length=hop_length)
    tempogram_ravel = np.ravel(tempogram)
    # Compute global onset autocorrelation
    ac_global = librosa.autocorrelate(oenv, max_size=tempogram.shape[0])
    ac_global = librosa.util.normalize(ac_global)   
    return mfccs, chroma_stft, mel,contrast,tonnetz, tempo_bpm, spectral_centroid, spec_bandwith, flatness, rolloff, zero_crossing_rate, oenv, tempogram_ravel, ac_global

def extract_audio_features_melspectrogram(audio_path): 
    y, sample_rate = librosa.load(audio_path)
    return y, sample_rate

In [ ]:
extract_audio_features(comedy_filename)

In [ ]:
def download(link, filelocation):
    r = requests.get(link, stream=True)
    with open(filelocation, 'wb') as f:
        for chunk in r.iter_content(1024):
            if chunk:
                f.write(chunk)
                
def createNewDownloadThread(link, filelocation):
    download_thread = threading.Thread(target=download, args=(link,filelocation))
    download_thread.start()

def download_poster(movie_data, _id, folder): 
    time.sleep(0.1)
    poster_url = movie_data[_id]['Poster']
    print('Poster URL:', poster_url)
    filename = poster_url[poster_url.rfind("/")+1:]
    print('filename:', filename)
    file_str = "/home/paulo/mestrado/posters/" + folder + "/" + _id + ".jpg"
    print('file:', file_str)    
    createNewDownloadThread(poster_url, file_str)

In [ ]:
def extract_statistics(feature_values):    
    _mean = np.mean(feature_values)
    _var = np.var(feature_values)
    _skew = scipy.stats.skew(feature_values)
    _kurtosis = scipy.stats.kurtosis(feature_values)
    _median = np.median(feature_values)
    _min = np.min(feature_values)
    _max = np.max(feature_values)
    return _mean, _var, _skew, _kurtosis, _median, _min, _max

In [ ]:

def process_movie(movie_data, trailer_labels, _id, i, Y, features, Genres, FullPlots, spectograms):        
    trailer_id = _id
    title_db = movie_data[_id]['Title']
    genres_db = movie_data[_id]['Genre']
    if 'FullPlot' in movie_data[_id].keys() and movie_data[_id]['FullPlot'] is not None:
          full_plot = movie_data[_id]['FullPlot']
    else:
          full_plot = movie_data[_id]['Plot']    
    labels = trailer_labels[i]                                
    avg_shot_length = 0
    color_var = 0
    light_key = 0
    mot_content = 0
    print((i, trailer_id, title_db, genres_db, labels, full_plot))
    Y[i] = labels
    Genres.append(genres_db)
    FullPlots.append(full_plot.lower())
    filename = 'trailers/videos/'+trailer_id+'.mp4'
    try:
            videodata = skvideo.io.vread(filename, num_frames=2000)
            #_, smooth_scene_cuts = shot_detection(videodata)
            scene_lum_idx = skvideo.measure.scenedet(videodata, method='histogram', parameter1=1.0)
            smooth_scene_cuts = videodata[scene_lum_idx]
            avg_shot_length = average_shot_length(videodata, smooth_scene_cuts)                
            color_var = color_variance(videodata, smooth_scene_cuts)
            light_key = lighting_key(videodata, smooth_scene_cuts)
            mot_content = motion_content(videodata, smooth_scene_cuts)        
            y, sr = extract_audio_features_melspectrogram(filename)   
            print(y.shape)
            spectograms[i] = y
            
            '''
            mfccs, chroma_stft, mel,contrast,tonnetz, tempo_bpm, spectral_centroid, spec_bandwith, flatness, rolloff, zero_crossing_rate, oenv, tempogram, ac_global = extract_audio_features(filename)                        
            mfccs_mean, mfccs_var, mfccs_skew, mfccs_kurtosis, mfccs_median, mfccs_min, mfccs_max = extract_statistics(mfccs)
            chroma_stft_mean, chroma_stft_var, chroma_stft_skew, chroma_stft_kurtosis, chroma_stft_median, chroma_stft_min, chroma_stft_max = extract_statistics(chroma_stft)        
            mel_mean, mel_var, mel_skew, mel_kurtosis, mel_median, mel_min, mel_max = extract_statistics(mel)
            contrast_mean, contrast_var, contrast_skew, contrast_kurtosis, contrast_median, contrast_min, contrast_max = extract_statistics(contrast)
            tonnetz_mean, tonnetz_var, tonnetz_skew, tonnetz_kurtosis, tonnetz_median, tonnetz_min, tonnetz_max = extract_statistics(tonnetz)
            spectral_centroid_mean, spectral_centroid_var, spectral_centroid_skew, spectral_centroid_kurtosis, spectral_centroid_median, spectral_centroid_min, spectral_centroid_max = extract_statistics(spectral_centroid)
            spec_bandwith_mean, spec_bandwith_var, spec_bandwith_skew, spec_bandwith_kurtosis, spec_bandwith_median, spec_bandwith_min, spec_bandwith_max = extract_statistics(spec_bandwith)
            flatness_mean, flatness_var, flatness_skew, flatness_kurtosis, flatness_median, flatness_min, flatness_max = extract_statistics(flatness)
            rolloff_mean, rolloff_var, rolloff_skew, rolloff_kurtosis, rolloff_median, rolloff_min, rolloff_max = extract_statistics(rolloff)
            zero_crossing_rate_mean, zero_crossing_rate_var, zero_crossing_rate_skew, zero_crossing_rate_kurtosis, zero_crossing_rate_median, zero_crossing_rate_min, zero_crossing_rate_max = extract_statistics(zero_crossing_rate)
            oenv_mean, oenv_var, oenv_skew, oenv_kurtosis, oenv_median, oenv_min, oenv_max = extract_statistics(oenv)
            tempogram_mean, tempogram_var, tempogram_skew, tempogram_kurtosis, tempogram_median, tempogram_min, tempogram_max = extract_statistics(tempogram)
            ac_global_mean, ac_global_var, ac_global_skew, ac_global_kurtosis, ac_global_median, ac_global_min, ac_global_max = extract_statistics(ac_global)                                                              
            
            features[i][0] = avg_shot_length
            features[i][1] = color_var
            features[i][2] = light_key
            features[i][3] = mot_content   
            #print('Video')
            features[i][4] = mfccs_mean
            features[i][5] = mfccs_var 
            features[i][6] = mfccs_skew
            features[i][7] = mfccs_kurtosis
            features[i][8] = mfccs_median
            features[i][9] = mfccs_min
            features[i][10] = mfccs_max
            #print('MFCC')
            features[i][11] = chroma_stft_mean
            features[i][12] = chroma_stft_var
            features[i][13] = chroma_stft_skew
            features[i][14] = chroma_stft_kurtosis
            features[i][15] = chroma_stft_median
            features[i][16] = chroma_stft_min
            features[i][17] = chroma_stft_max
            #print('Chroma')
            features[i][18] = mel_mean
            features[i][19] = mel_var
            features[i][20] = mel_skew
            features[i][21] = mel_kurtosis
            features[i][22] = mel_median
            features[i][23] = mel_min
            features[i][24] = mel_max
            #print('Mel')
            features[i][25] = contrast_mean
            features[i][26] = contrast_var
            features[i][27] = contrast_skew
            features[i][28] = contrast_kurtosis
            features[i][29] = contrast_median
            features[i][30] = contrast_min
            features[i][31] = contrast_max
            #print('Contrast')
            features[i][32] = tonnetz_mean
            features[i][33] = tonnetz_var
            features[i][34] = tonnetz_skew
            features[i][35] = tonnetz_kurtosis
            features[i][36] = tonnetz_median
            features[i][37] = tonnetz_min
            features[i][38] = tonnetz_max
            #print('Tonnetz')
            features[i][39] = tempo_bpm 
            #print('Tempo')
            features[i][40] = spectral_centroid_mean
            features[i][41] = spectral_centroid_var
            features[i][42] = spectral_centroid_skew
            features[i][43] = spectral_centroid_kurtosis
            features[i][44] = spectral_centroid_median
            features[i][45] = spectral_centroid_min
            features[i][46] = spectral_centroid_max
            #print('Spectral Centroid')
            features[i][47] = spec_bandwith_mean
            features[i][48] = spec_bandwith_var
            features[i][49] = spec_bandwith_skew
            features[i][50] = spec_bandwith_kurtosis
            features[i][51] = spec_bandwith_median
            features[i][52] = spec_bandwith_min
            features[i][53] = spec_bandwith_max            
            #print('Spectral Bandwith')
            features[i][54] = flatness_mean
            features[i][55] = flatness_var
            features[i][56] = flatness_skew
            features[i][57] = flatness_kurtosis
            features[i][58] = flatness_median
            features[i][59] = flatness_min
            features[i][60] = flatness_max
            #print('Flatness')
            features[i][61] = rolloff_mean
            features[i][62] = rolloff_var
            features[i][63] = rolloff_skew
            features[i][64] = rolloff_kurtosis
            features[i][65] = rolloff_median
            features[i][66] = rolloff_min
            features[i][67] = rolloff_max
            #print('Rolloff')
            features[i][68] = zero_crossing_rate_mean
            features[i][69] = zero_crossing_rate_var
            features[i][70] = zero_crossing_rate_skew
            features[i][71] = zero_crossing_rate_kurtosis
            features[i][72] = zero_crossing_rate_median
            features[i][73] = zero_crossing_rate_min            
            features[i][74] = zero_crossing_rate_max
            #print('Zero Crossing Rate')
            features[i][75] = oenv_mean
            features[i][76] = oenv_var
            features[i][77] = oenv_skew
            features[i][78] = oenv_kurtosis
            features[i][79] = oenv_median
            features[i][80] = oenv_min
            features[i][81] = oenv_max
            #print('Oenv')
            features[i][82] = tempogram_mean
            features[i][83] = tempogram_var
            features[i][84] = tempogram_skew
            features[i][85] = tempogram_kurtosis
            features[i][86] = tempogram_median
            features[i][87] = tempogram_min
            features[i][88] = tempogram_max
            #print('Tempogram')
            features[i][89] = ac_global_mean
            features[i][90] = ac_global_var
            features[i][91] = ac_global_skew
            features[i][92] = ac_global_kurtosis
            features[i][93] = ac_global_median
            features[i][94] = ac_global_min
            features[i][95] = ac_global_max        
            #print('Ac Global')
            '''
            del videodata
            del smooth_scene_cuts                    
    except Exception as e:
        print("error: {0}".format(e))
        pass    

def extract_XY(trailer_ids, trailer_labels, folder, num_features = 4, num_genres = 9): #num_features = 96
    n_cpus = multiprocessing.cpu_count()    
    print('n_cpus:',n_cpus)      
    Genres = []    
    FullPlots = [] 
    max_movies = min(10,len(trailer_ids))
    poster_images = np.zeros((max_movies, width, height, depth))
    spectograms = np.zeros((max_movies, audio_channels, sr))
    Y = np.zeros((max_movies, num_genres))  #  
    features = np.zeros((max_movies, num_features))
    movie_data = lmtd.get_data_by_trailer_ids(trailer_ids)
    i=0
    for _id in trailer_ids:  
        if i < max_movies:     
            print(i)                        
            process_movie(movie_data, trailer_labels, _id, i, Y, features, Genres, FullPlots, spectograms) 
            img_path = "/home/paulo/mestrado/posters/" + folder + "/" + _id + ".jpg"
            try:
                pil_img = image.load_img(img_path, target_size=(299,299))        
                poster_images[i] = image.img_to_array(pil_img)             
            except:
                pass
        i+=1            
    X = features
    print(X.shape)    
    print(X)
    print(Y.shape)
    print(Y)
    print(Genres)
    return X, Y, Genres, FullPlots, poster_images, spectograms

In [ ]:
training_trailer_ids = lmtd.train_ids
validation_trailer_ids = lmtd.valid_ids
test_trailer_ids = lmtd.test_ids
all_ids = list()
all_ids.extend(training_trailer_ids)
all_ids.extend(validation_trailer_ids)
all_ids.extend(test_trailer_ids)
all_ids = set(all_ids)
print('Set length movie_trailers:', len(all_ids))

all_movie_titles = set()

i = 0
with tqdm(total=len(all_ids), file=sys.stdout) as pbar:
        movie_data = lmtd.get_data_by_trailer_ids(all_ids)
        for _id in all_ids:    
            pbar.set_description('processed: %d' % (1 + i))
            pbar.update(1)
            # Returns a dictionary in which keys are the queried trailer_ids            
            title_db = movie_data[_id]['Title']
            all_movie_titles.add(title_db)
        i+=1
        
print('Diferent movie titles:', len(all_movie_titles))
print(all_movie_titles)

In [ ]:
train_X, train_Y, train_Genres, train_FullPlots, train_poster_images, train_spectograms = extract_XY(training_trailer_ids, lmtd.train_labels, 'train')

In [ ]:
print(train_spectograms.shape)

# Display Correlations among Variables and Output Class

In [ ]:
sns.set(style="ticks")

name_to_index = {8: 'Action', 7: 'Adventure', 6: 'Comedy', 5: 'Crime', 4: 'Drama', 3: 'Horror', 2: 'Romance', 1: 'SciFi', 0: 'Thriller'}
train_Genres_encoded = []
for i in range(train_Y.shape[0]):
    genres_in_movie = (train_Y[i] == 1)
    genre_indexes = [i for i, x in enumerate(genres_in_movie) if x]
    #print(genre_indexes)
    genres_in_movie_dict = {k: v for k, v in name_to_index.items() if k in genre_indexes and len(genre_indexes)==1}    
    genres_in_movie_dict = OrderedDict(sorted(genres_in_movie_dict.items(), key=operator.itemgetter(1)))    
    genres_in_movie_str = ','.join(genres_in_movie_dict.values())
    print(genres_in_movie_str)
    #print(name_to_index[0])
    train_Genres_encoded.append(genres_in_movie_str)
  
d = {'avg_shot_length': train_X[:,0], 
     'color_var': train_X[:,1], 
     'light_key': train_X[:,2], 
     'mot_content': train_X[:,3], 
     'mfccs_mean': train_X[:,4], 
     #'chroma_stft_mean': train_X[:,11], 
     'mel_mean': train_X[:,18], 
     #'contrast_mean': train_X[:,25], 
     #'tonnetz_mean': train_X[:,32], 
     'tempo_bpm': train_X[:,39],
     #'spectral_centroid_mean': train_X[:,40],
     #'spec_bandwith_mean': train_X[:,47],
     'flatness_mean': train_X[:,54],
     #'rolloff_mean': train_X[:,61],
     'zero_crossing_rate_mean': train_X[:,68],
     #'oenv_mean': train_X[:,75],
     #'tempogram_mean': train_X[:,82],
     #'ac_global_mean': train_X[:,89],
     'genres': train_Genres_encoded}
print(d)
df = pd.DataFrame(data=d)
print(df.head())
sns.pairplot(df, hue="genres")

In [ ]:
val_X, val_Y, val_Genres, val_FullPlots, val_poster_images, val_spectograms = extract_XY(validation_trailer_ids, lmtd.valid_labels, 'valid')

In [ ]:
test_X, test_Y, test_Genres, test_FullPlots, test_poster_images, test_spectograms = extract_XY(test_trailer_ids, lmtd.test_labels, 'test')

# Normalization

In [ ]:
scaler = preprocessing.StandardScaler()
train_X = scaler.fit_transform(train_X)
val_X = scaler.transform(val_X)
test_X = scaler.transform(test_X)

In [ ]:
le = preprocessing.LabelEncoder()
train_Genres_encoded = le.fit_transform(train_Genres) 
val_Genres_encoded = le.fit_transform(val_Genres) 
test_Genres_encoded = le.fit_transform(test_Genres) 

In [ ]:
print(train_Genres_encoded)

In [ ]:
print(val_Genres_encoded)

In [ ]:
print(test_Genres_encoded)

# Select Features on validation

In [ ]:
#select features
feature_selection = ExtraTreesClassifier()
feature_selection = feature_selection.fit(val_X, val_Y)
print(feature_selection.feature_importances_)
importances = feature_selection.feature_importances_
std = np.std([tree.feature_importances_ for tree in feature_selection.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

doFeatureSelection = False
if doFeatureSelection:
    model = SelectFromModel(feature_selection, prefit=True)
    train_X = model.transform(train_X)
    val_X = model.transform(val_X)
    test_X = model.transform(test_X)
    print(train_X.shape)
    print(val_X.shape)
    print(test_X.shape)


# Print the feature ranking
print("Feature ranking:")

for f in range(val_X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))    
    
# Plot the feature importances of the forest
plt.figure(figsize=(50,50))
plt.title("Feature importances")
plt.bar(range(val_X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(val_X.shape[1]), indices)
plt.xlim([-1, val_X.shape[1]])
plt.show()

# Find best parameters on validation

In [ ]:
print(len(val_Genres_encoded))
class_str = 'SVC'
stratified = len(val_Genres_encoded) > 100

C_range = 10. ** np.arange(-3, 3)
if 'SVC' in class_str:    
    gamma_range = 10. ** np.arange(-5, 4)
    param_grid = dict(gamma=gamma_range, C=C_range, kernel=['linear', 'rbf'])
elif 'Logistic' in class_str:
    param_grid = dict(C=C_range)

if stratified:
    validation = StratifiedKFold(n_splits=2)
else:
    validation = KFold(n_splits=2) 

if 'SVC' in class_str:    
    grid = GridSearchCV(SVC(), param_grid=param_grid, cv=validation, n_jobs=6)
elif 'Logistic' in class_str:
    grid = GridSearchCV(LogisticRegression(), param_grid=param_grid, cv=validation, n_jobs=6)
grid.fit(val_X, val_Genres_encoded)

print("Os melhores parametros %s com um score de %0.2f" % (grid.best_params_, grid.best_score_))

# Classify

In [ ]:
#classify
C_best_param = grid.best_params_['C']
if 'SVC' in class_str:    
    kernel_best_param=grid.best_params_['kernel']
    gamma_best_param = grid.best_params_['gamma']
    classifier = SVC(C=C_best_param, gamma=gamma_best_param, kernel=kernel_best_param, class_weight='balanced', random_state=13)
elif 'Logistic' in class_str:    
    classifier = LogisticRegression(solver='sag', max_iter=100, C=C_best_param, random_state=13, class_weight='balanced', multi_class='ovr', n_jobs=6)

ovr = OneVsRestClassifier(classifier)
ovr.fit(train_X, train_Y)
test_Ypred = ovr.predict(test_X)
print(test_Ypred)

In [ ]:
print('Average precision micro,macro, weighted:', evaluation.prauc(test_Y,test_Ypred))

In [ ]:
genres_names = ['Action', 'Adventure', 'Comedy', 'Crime', 'Drama', 'Horror', 'Romance', 'SciFi', 'Thriller']
print(genres_names)
print(np.sum(test_Y, axis=0))
print(np.sum(test_Ypred, axis=0))

df = {'genres': genres_names, 'true': np.sum(test_Y, axis=0), 'pred': np.sum(test_Ypred, axis=0)}

fig, axs = plt.subplots(figsize=(20,15), ncols=2)
ax = sns.barplot(x='genres', y='true', data=df, ax=axs[0])
ax.set_title("True Labels")
ax.set_xlabel('genres')

sns.barplot(x='genres', y='pred', data=df, ax=axs[1]).set_title("Pred Labels")
plt.show()

# Roc 

In [ ]:
lw = 2

n_classes = test_Y.shape[1]
y_score = ovr.fit(train_X, train_Y).decision_function(test_X)
print(n_classes)
print(test_Genres)
print(test_Y.shape)
print(y_score.shape)
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(test_Y[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
#y_score_max = np.argmax(y_score, axis=1)
fpr["micro"], tpr["micro"], _ = roc_curve(test_Y.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

# Deep Learning Keras

# SoftAttention Layer

In [ ]:
def make_safe(x):
    return K.clip(x, K.common._EPSILON, 1.0 - K.common._EPSILON)

class Wrapper(Layer):
    def __init__(self, layer, **kwargs):
        self.layer = layer
        super(Wrapper, self).__init__(**kwargs)

    def build(self, input_shape):       
        super(Wrapper, self).build(input_shape)  # Be sure to call this somewhere!   
       
    def call(self, x):
        super(Wrapper, self).call(x)

class ProbabilityTensor(Wrapper):
    """ function for turning 3d tensor to 2d probability matrix, which is the set of a_i's """
    def __init__(self, dense_function=None, *args, **kwargs):
        self.supports_masking = True
        self.input_spec = [tf.keras.layers.InputSpec(ndim=3)]
        #layer = TimeDistributed(dense_function) or TimeDistributed(Dense(1, name='ptensor_func'))
        layer = TimeDistributed(Dense(1, name='ptensor_func'))
        super(ProbabilityTensor, self).__init__(layer, *args, **kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.input_spec = [tf.keras.layers.InputSpec(shape=input_shape)]
        if K._BACKEND == 'tensorflow':
            if not input_shape[1]:
                raise Exception('When using TensorFlow, you should define '
                                'explicitly the number of timesteps of '
                                'your sequences.\n'
                                'If your first layer is an Embedding, '
                                'make sure to pass it an "input_length" '
                                'argument. Otherwise, make sure '
                                'the first layer has '
                                'an "input_shape" or "batch_input_shape" '
                                'argument, including the time axis.')

        if not self.layer.built:
            self.layer.build(input_shape)
            self.layer.built = True
        super(ProbabilityTensor, self).build(input_shape)

    def compute_output_shape(self, input_shape):
        # b,n,f -> b,n
        #       s.t. \sum_n n = 1
        if isinstance(input_shape, (list,tuple)) and not isinstance(input_shape[0], int):
            input_shape = input_shape[0]

        return (input_shape[0], input_shape[1])

    def squash_mask(self, mask):
        if K.ndim(mask) == 2:
            return mask
        elif K.ndim(mask) == 3:
            return K.any(mask, axis=-1)

    def compute_mask(self, x, mask=None):
        if mask is None:
            return None
        return self.squash_mask(mask)

    def call(self, x, mask=None):
        energy = K.squeeze(self.layer(x), 2)
        p_matrix = K.softmax(energy)
        if mask is not None:
            mask = self.squash_mask(mask)
            p_matrix = make_safe(p_matrix * mask)
            p_matrix = (p_matrix / K.sum(p_matrix, axis=-1, keepdims=True))*mask
        return p_matrix

    def get_config(self):
        config = {}
        base_config = super(ProbabilityTensor, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

class SoftAttentionConcat(ProbabilityTensor):
    '''This will create the context vector and then concatenate it with the last output of the LSTM'''
    def compute_output_shape(self, input_shape):
        # b,n,f -> b,f where f is weighted features summed across n
        return (input_shape[0], 2*input_shape[2])

    def compute_mask(self, x, mask=None):
        if mask is None or mask.ndim==2:
            return None
        else:
            raise Exception("Unexpected situation")

    def call(self, x, mask=None):
        # b,n,f -> b,f via b,n broadcasted
        p_vectors = K.expand_dims(super(SoftAttentionConcat, self).call(x, mask), 2)
        expanded_p = K.repeat_elements(p_vectors, K.int_shape(x)[2], axis=2)
        context = K.sum(expanded_p * x, axis=1)
        last_out = x[:, -1, :]
        return K.concatenate([context, last_out])

In [ ]:
all_data = []
all_data.extend(train_FullPlots)
all_data.extend(val_FullPlots)
all_data.extend(test_FullPlots)
print(all_data)

In [ ]:
tokenizer = setup_tokenizer(all_data)
word_index = tokenizer.word_index
embeddings_index = setup_glove_embeddings()
embedding_matrix = setup_embeddings_matrices(embeddings_index, word_index)
embedding_layer = setup_embedding_layer(word_index, embedding_matrix)

In [ ]:
print('Pad sequences (samples x time)')
X_train_text = sequence.pad_sequences(tokenizer.texts_to_sequences(train_FullPlots), maxlen=maxlen)
X_val_text = sequence.pad_sequences(tokenizer.texts_to_sequences(val_FullPlots), maxlen=maxlen)
X_test_text = sequence.pad_sequences(tokenizer.texts_to_sequences(test_FullPlots), maxlen=maxlen)

In [ ]:
print('x_train shape:', X_train_text.shape)
print('x_val shape:', X_val_text.shape)
print('x_test shape:', X_test_text.shape)

# Branch 1 - RNN for Text (Full Plot)

In [ ]:
sequence_input = Input(shape=(max_sentence_len,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

# RNN
lstm = Bidirectional(LSTM(lstm_size, return_sequences=True, recurrent_dropout=recurrent_dropout))(embedded_sequences)
attention_1 = SoftAttentionConcat()(lstm)
batch1 = BatchNormalization()(attention_1)

branch_1 = Dense(512, activation='relu')(batch1)

# Branch 2 - CNN for images (Poster)

In [ ]:
# Image input branch - a pre-trained Inception module followed by an added fully connected layer
base_model = applications.InceptionV3(weights='imagenet', include_top=False)
# Freeze Inception's weights - we don't want to train these
for layer in base_model.layers:
    layer.trainable = False

# add a fully connected layer after Inception - we do want to train these
branch_2 = base_model.output
branch_2 = GlobalAveragePooling2D()(branch_2)
branch_2 = Dense(512, activation='relu')(branch_2)

# Branch 3 - CNN for spectogram image (Audio)

In [ ]:
# A mel-spectrogram layer
audio_inputs = Input(shape=audio_input_shape)
branch3 = Melspectrogram(n_dft=512, n_hop=256,
                         padding='same', sr=sr, n_mels=128,
                         fmin=0.0, fmax=sr/2, power_melgram=1.0,
                         return_decibel_melgram=False, trainable_fb=False,
                         trainable_kernel=False,
                         name='trainable_stft')(audio_inputs)
# Maybe some additive white noise.
branch3 = AdditiveNoise(power=0.2)(branch3)
# If you wanna normalise it per-frequency
branch3 = Normalization2D(str_axis='freq')(branch3) # or 'channel', 'time', 'batch', 'data_sample'
#CNN
branch3 = Conv2D(32, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform')(branch3)
branch3 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform')(branch3)
branch3 = BatchNormalization()(branch3)
branch3 = MaxPooling2D(pool_size=(2, 2))(branch3)
branch3 = Dropout(0.25)(branch3)
branch3 = Conv2D(64, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform')(branch3)
branch3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform')(branch3)
branch3 = BatchNormalization()(branch3)
branch3 = MaxPooling2D(pool_size=(2, 2))(branch3)
branch3 = Dropout(0.25)(branch3)
branch3 = Conv2D(64, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform')(branch3)
branch3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform')(branch3)
branch3 = BatchNormalization()(branch3)
branch3 = MaxPooling2D(pool_size=(2, 2))(branch3)
branch3 = Dropout(0.25)(branch3)
branch3 = GlobalAveragePooling2D()(branch3)
branch3 = Dense(512, activation='relu')(branch3)

# Branch 4 - LTMD9 Features 

In [ ]:
# Be sure to update your LMTD_PATH
'''
LMTD_PATH = '/home/paulo/mestrado/lmtd'
features_path = os.path.join(LMTD_PATH, 'features', 'lmtd9_resnet152.pickle')
lmtd.load_precomp_features(features_file=features_path)

x_valid_lmtd, x_valid_len, y_valid_lmtd, valid_ids = lmtd.get_split('valid')
x_train_lmtd, x_train_len, y_train_lmtd, train_ids = lmtd.get_split('train')
x_test_lmtd,  x_test_len,  y_test_lmtd,  test_ids  = lmtd.get_split('test')

inputs = Input(shape=(time_steps, nb_features))
branch4 = BatchNormalization()(inputs)
branch4 = Convolution1D(conv_filters, kernel_size=3)(branch4)
branch4 = BatchNormalization()(branch4)
branch4 = Activation('relu')(branch4)
branch4 = GlobalMaxPooling1D()(branch4)
branch4 = Dropout(dropout)(branch4)
branch4 = Dense(512, activation='relu')(branch4)
'''

# join branches

In [ ]:
num_classes = np.max(train_Genres_encoded) + 1
print(num_classes)

y_train = to_categorical(train_Genres_encoded, num_classes=num_classes)
y_val = to_categorical(val_Genres_encoded, num_classes=num_classes)
y_test = to_categorical(val_Genres_encoded, num_classes=num_classes)

print(y_train)
print(y_val)
print(y_test)

In [ ]:
# merge the text input branch and the image input branch and add another fully connected layer
joint = merge([branch_1, branch_2, branch3], mode='concat')
joint = Dense(2048, activation='relu')(joint)
joint = Dropout(0.5)(joint)
predictions = Dense(num_classes, activation='softmax', kernel_initializer='glorot_uniform')(joint)

full_model = Model(inputs=[base_model.input, sequence_input, audio_inputs], outputs=[predictions])

In [ ]:
full_model.summary()

In [ ]:
start_time = datetime.datetime.now()
print('Start:',start_time)

#TIP: set tensorflow number of threads (check number of cores in CPU and use half of maximum)
number_gpus = number_gpus
number_cpus = number_cpus
with tf.Session(config = tf.ConfigProto(device_count={'GPU': number_gpus, 'CPU': number_cpus}, log_device_placement=True)) as sess:
    K.set_session(sess)
    
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    optimizer = Adam(lr=learning_rate)
    full_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    history = full_model.fit([train_poster_images, X_train_text, train_spectograms], y_train,
                         epochs=max_epochs, batch_size=batch_size, verbose=1, 
                         validation_data=([val_poster_images, X_val_text, val_spectograms], y_val), 
                         shuffle=True)
    
    full_model.save('MovieGenreClassifier.h5')
    
    y_pred_test = model.predict([test_poster_images, X_test_text, test_spectograms])    
    result = evaluation.prauc(y_test, y_pred_test)
    
    print_result(result)
        
end_time = datetime.datetime.now()
print('End:',start_time)